# Table of Contents

- [Overview](#overview)
- [Imports & Setup](#imports--setup)
- [Data Loading](#data-loading)
- [Preprocessing & Feature Engineering](#preprocessing--feature-engineering)
- [Evaluation](#evaluation)


# Imports & Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import resample

# Data Loading

In [ ]:
data = pd.read_excel(r"path")
data.info()

# Preprocessing & Feature Engineering

In [ ]:
test_mapping_dict = {
    'ct': 'ct',
    None: 'unknown',
    'ct angiography': 'angiography',
    'ct arthrography': 'ct',
    'ct function and morphology': 'ct',
    'ct maxillofacial': 'ct',
    'ct maxface': 'ct',
    'ct venography': 'ct',
    'ct angiography-perfusion': 'angiography',
    'ct colonography': 'ct',
    'ct enterography': 'ct',
    'lumbar nerve root block': 'nerve injection',
    'xray': 'xray',
    'ct high resolution': 'ct',
    'ct quantitative': 'ct',
    'ct urography': 'ct',
    'mri': 'mri',
    'ultrasound': 'ultrasound',
    'follow up ': 'follow-up',
    'intestine': 'gi imaging',
    'epidural steroid injection': 'nerve injection',
    'ct, xray': 'ct',
    'ct ': 'ct',
    'ct colonography ': 'ct',
    'ovarian': 'ovarian',
    ' ct angiography': 'angiography',
    'ct function and morphology ': 'ct',
    'ct enteroclysis': 'ct',
    'petct': 'pet-ct',
    'colonography ': 'gi imaging',
    '  ct angiography': 'angiography',
    'ct head venography': 'ct',
    'ct cystography': 'ct',
    'ct maxface ': 'ct',
    'nuclear bone scan': 'nuclear scan',
    'biopsy': 'biopsy',
    'ct functionand morphology': 'ct'
}


In [ ]:
test_mapping_dict = {
    'ct': 'ct',
    None: 'unknown',
    'ct angiography': 'angiography',
    'ct arthrography': 'ct',
    'ct function and morphology': 'ct',
    'ct maxillofacial': 'ct',
    'ct maxface': 'ct',
    'ct venography': 'ct',
    'ct angiography-perfusion': 'angiography',
    'ct colonography': 'ct',
    'ct enterography': 'ct',
    'lumbar nerve root block': 'nerve injection',
    'xray': 'xray',
    'ct high resolution': 'ct',
    'ct quantitative': 'ct',
    'ct urography': 'ct',
    'mri': 'mri',
    'ultrasound': 'ultrasound',
    'follow up ': 'follow-up',
    'intestine': 'gi imaging',
    'epidural steroid injection': 'nerve injection',
    'ct, xray': 'ct',
    'ct ': 'ct',
    'ct colonography ': 'ct',
    'ovarian': 'ovarian',
    ' ct angiography': 'angiography',
    'ct function and morphology ': 'ct',
    'ct enteroclysis': 'ct',
    'petct': 'pet-ct',
    'colonography ': 'gi imaging',
    '  ct angiography': 'angiography',
    'ct head venography': 'ct',
    'ct cystography': 'ct',
    'ct maxface ': 'ct',
    'nuclear bone scan': 'nuclear scan',
    'biopsy': 'biopsy',
    'ct functionand morphology': 'ct'
}

data['clinician_exam_flags'] = data['clinician_exam_flags'].fillna(-1)
data['claude_exam_flags'] = data['claude_exam_flags'].fillna(-1)
data['chatGPT_exam_flags'] = data['chatGPT_exam_flags'].fillna(-1)
data['clinician_exam_flags'] = data['clinician_exam_flags'].astype('int64')
data['claude_exam_flags'] = data['claude_exam_flags'].astype('int64')
data['chatGPT_exam_flags'] = data['chatGPT_exam_flags'].astype('int64')

In [ ]:

modalities = [ 
    'ct', np.nan, 'ct angiography', 'ct arthrography',
    'ct function and morphology', 'ct maxillofacial', 'ct maxface',
    'ct venography', 'ct angiography-perfusion', 'ct colonography',
    'ct enterography', 'lumbar nerve root block', 'xray',
    'ct high resolution', 'ct quantitative', 'ct urography', 'mri',
    'ultrasound', 'follow up ', 'intestine',
    'epidural steroid injection', 'ct, xray', 'ct ',
    'ct colonography ', 'ovarian', ' ct angiography',
    'ct function and morphology ', 'ct enteroclysis', 'petct',
    'colonography ', '  ct angiography', 'ct head venography',
    'ct cystography', 'ct maxface ', 'nuclear bone scan', 'biopsy',
    'ct functionand morphology'
]

# Define mapping function
def map_to_cy(value):
    """
    Map exam / modality strings to one of: 'angiography', 'ct', 'other'.
    Anything unparseable becomes np.nan.
    """
    # Handle missing
    if pd.isna(value):
        return np.nan

    # Coerce everything to str *safely*
    try:
        text = str(value).strip().lower()
    except Exception:          # e.g. list, dict
        return np.nan

    if "angiography" in text:
        return "angiography"
    elif "ct" in text:
        return "ct"
    else:
        return "other"


# Apply mapping
mapped_modalities = list(map(map_to_cy, modalities))

data['test_type'] = data['ESR_iGuide_exam'].map(map_to_cy)

In [ ]:

# Define the clusters and the tokenize_and_flag function as we established above
clusters = {
    "head": ["head", "cranial", "skull", "brain", "cerebral", "facial", "sinus", "paranasal sinuses", "temporal bone", 
             "face", "orbit", "temporomandibular joints", "maxilla", "sinuses", "mandible", "pituitary gland", 
             "maxillofacial area", "maxillofacial", "nasopharynx",  "salivary glands", "maxillofacial region", 
             "mouth area", 'tongue', 'scalp', 'pituitary', 'eye', 'intracranial', "ear", "temporomandibular joint"],
    "neck": ["neck", "cervical", "throat", "nuchal", "larynx", "esophagus", "carotid arteries", "carotid", "parotid gland", "thyroid"],
    "thorax": ["thorax", "chest", "thoracic", "pulmonary", "heart", "cardiac", "breast", "mediastinum", "aorta", 
               "aortic", "coronary", "coronaries", "aorta branches", "sternoclavicular joint", "breasts", "trachea", "lung", "lungs", "clavicula", "scapula bone",
               "joint sternoclavicular", "subclavian artery", "coronary arteries"],
    "abdomen_pelvis": ["abdomen", "abdominal", "stomach", "intestinal", "gastrointestinal", "liver", "pancreas", "spleen", "small bowel", "colon", 
                       "colonography colon", "gallbladder", "kidney", "kidneys", "urinary organs", "biliary tract", "biliary system", "renal", "adrenal",
                       "adrenal gland", "adrenal glands", "pelvis", "pelvic", "hip", "inguinal", "pubic", "iliac vein", "urinary bladder", "urinary tract", "prostate",
                       "uterus", "uterus and ovaries"],
    "upper_extremities": ["upper", "arm", "shoulder", "elbow", "wrist", "hand", "scapula", "clavicle", 
                          "humerus", "right humerus", "ulna left", "forearm", "left forearm", "finger", 'brachial plexus', "right thumb"],
    "lower_extremities": ["lower", "leg", "knee", "knees", "foot", "thigh", "tibia", "femur", "calcaneus", 
                          "popliteal artery", "knees bilateral", "right malleolus", 'femoral nerve left', "lower extremities", "iliofemoral arteries", "ankle"],
    "spine": ["spine", "vertebral", "lumbar", "sacral", "spinal canal", "spinal cord", "spinal", "thoracic spine"],
    "skeletal": ["joint", "bone", "bones", "skeletal", "skeleton", "extremities", "musculoskeletal", "musculoskeletal system"],
    "lymphatic": ["lymph nodes", "lymphatic system"],
    "body": ["whole body", "body", 'full body', 'various organs', 'multiple organs',
              "extremities", "multiple organ systems", 'muscular system', 'skin', 'limbs', "blood", "peripheral", 'endocrine system', "muscles",
              "vascular region", "vascular system", "arterial system"]
}

def tokenize_and_flag(phrase):
    if pd.isna(phrase):
        return None
    cleaned_phrase = re.sub(r"-", " ", str(phrase))
    cleaned_phrase = re.sub(r"[^\w\s]", "", cleaned_phrase.lower())
    binary_flag = 0
    cluster_tokens = {
        "head": 1 << 0, "neck": 1 << 1, "thorax": 1 << 2, "abdomen_pelvis": 1 << 3,
        "upper_extremities": 1 << 4, "lower_extremities": 1 << 5, "spine": 1 << 6,
        "skeletal": 1 << 7, "lymphatic": 1 << 8, "body": 1 << 9
    }
    for cluster, terms in clusters.items():
        if any(re.search(r"\b" + re.escape(term) + r"\b", cleaned_phrase) for term in terms):
            binary_flag |= cluster_tokens[cluster]
    return binary_flag



data['clinician_organ_flags'] = data['clinician_organ'].apply(tokenize_and_flag)
data['ESR_iGuide_organ_flags'] = data['ESR_iGuide_organ'].apply(tokenize_and_flag)
data['claude_organ_flags'] = data['claude_organ'].apply(tokenize_and_flag)
data['chatGPT_organ_flags'] = data['chatGPT_organ'].apply(tokenize_and_flag)

data['clinician_organ_flags'] = data['clinician_organ_flags'].fillna(-1)
data['claude_organ_flags'] = data['claude_organ_flags'].fillna(-1)
data['chatGPT_organ_flags'] = data['chatGPT_organ_flags'].fillna(-1)

data['clinician_organ_flags'] = data['clinician_organ_flags'].astype('int64')
data['claude_organ_flags'] = data['claude_organ_flags'].astype('int64')
data['chatGPT_organ_flags'] = data['chatGPT_organ_flags'].astype('int64')


data = data.dropna(subset=['ESR_iGuide_contrast'])

In [ ]:
# Define a mapping for standardizing contrast descriptions
contrast_columns = ['ESR_iGuide_contrast', 'clinician_contrast', 'claude_contrast', 'chatGPT_contrast']

contrast_standardization_map = {
'with iv contrast ': 'with iv contrast', 
'without iv contrast '
'without iv contrast ':'without iv contrast',
'without iv contrast_x000D_':'without iv contrast',
'no iv contrast':'without iv contrast',
' with iv contrast': 'with iv contrast',
 'with iv contrast ':'with iv contrast',
 
       ' with or without iv contrast':'with or without iv contrast', 
       ' without iv contrast': 'without iv contrast',
       'with or without iv contrast ':'with or without iv contrast',
       ' with iv contrast': 'with iv contrast',
       'with or without iv contrast ':'with or without iv contrast', 
       ' with or without iv contrast': 'with or without iv contrast',
       ' without iv contrast': 'without iv contrast',
       'wo /w iv contrast': 'with or without iv contrast',
       'w iv contras': 'with iv contrast',
     'w iv contrast': 'with iv contrast', 
     'wo/w iv contrast': 'with or without iv contrast',
       'wo/with iv contrast': 'with or without iv contrast',
         'wo /w iv contrast': 'with or without iv contrast',
       
       'w/without iv contrast': 'with or without iv contrast',
        'wo iv. contrast' :'without iv contrast',
       'wo iv contarast': 'without iv contrast',
         'w contrast':'with iv contrast',
           'w iv contrast  ':'with iv contrast',
       'wo in contrast':'with iv contrast',
       'w iv contrast ':'without iv contrast', 
       'wo/w. iv contrast': 'with or without iv contrast',
       'w i.v. contrast':'with iv contrast', 
       'w iv contrat':'with iv contrast',
         'wo contrast':'without iv contrast',
       'w. iv contrast':'with iv contrast',
       'wo i.v. contrast':'without iv contrast',
       'wo/w i.v. contrast': 'with or without iv contrast',
       'wo/w iv contrast ': 'with or without iv contrast',
       'w ic contrast ':'with iv contrast',
       'without iv contrast ':'without iv contrast'




}

# Function to apply standardization
def standardize_contrast(column):
    return column.map(contrast_standardization_map).fillna(column)

# Apply the standardization function to the contrast columns
for col in contrast_columns:
    data[col] = standardize_contrast(data[col])


def encode_binary_flags(df, column):
    df[column + '_flags'] = df[column].map({
        'with iv contrast': 1,
        'without iv contrast': 0,
        'with or without iv contrast': 2  # or another suitable encoding
    })

for col in [ 'ESR_iGuide_contrast', 'clinician_contrast','claude_contrast', 'chatGPT_contrast']:
    encode_binary_flags(data, col)



data['clinician_contrast_flags'] = data['clinician_contrast_flags'].fillna(-1)
data['claude_contrast_flags'] = data['claude_contrast_flags'].fillna(-1)
data['chatGPT_contrast_flags'] = data['chatGPT_contrast_flags'].fillna(-1)


data['clinician_contrast_flags'] = data['clinician_contrast_flags'].astype('int64')
data['claude_contrast_flags'] = data['claude_contrast_flags'].astype('int64')
data['chatGPT_contrast_flags'] = data['chatGPT_contrast_flags'].astype('int64')

# Evaluation

In [ ]:


def group_permutation_p_value(y_true_a, y_pred_a,
                              y_true_b, y_pred_b,
                              metric_func=accuracy_score,
                              n_permutations=1000):
    """
    Permutation p-value for the difference of `metric_func`
    between subset A and subset B.
    """
    # ❶ Drop -1 labels
    m_a = (y_true_a != -1) & (y_pred_a != -1)
    m_b = (y_true_b != -1) & (y_pred_b != -1)
    y_true_a, y_pred_a = y_true_a[m_a], y_pred_a[m_a]
    y_true_b, y_pred_b = y_true_b[m_b], y_pred_b[m_b]

    if len(y_true_a) == 0 or len(y_true_b) == 0:
        return np.nan          # nothing to compare

    # ❷ Observed gap (B – A)
    obs_gap = metric_func(y_true_b, y_pred_b) - metric_func(y_true_a, y_pred_a)

    # ❸ Pool and permute
    y_true_pool = np.concatenate([y_true_a, y_true_b])
    y_pred_pool = np.concatenate([y_pred_a, y_pred_b])
    n_a = len(y_true_a)

    hits = 0
    for _ in range(n_permutations):
        perm_idx = np.random.permutation(len(y_true_pool))
        a_idx, b_idx = perm_idx[:n_a], perm_idx[n_a:]
        perm_gap = (metric_func(y_true_pool[b_idx], y_pred_pool[b_idx]) -
                    metric_func(y_true_pool[a_idx], y_pred_pool[a_idx]))
        if abs(perm_gap) >= abs(obs_gap):
            hits += 1

    return hits / n_permutations


#### by sex

In [ ]:
# --------------------------------------------------------------
# 0 .  Setup  (make sure these columns exist in exam_data)
# --------------------------------------------------------------
entries = {
    'exam': ['ESR_iGuide_exam_flags','clinician_exam_flags','chatGPT_exam_flags','claude_exam_flags'], 
    'organ':['ESR_iGuide_organ_flags','clinician_organ_flags','chatGPT_organ_flags','claude_organ_flags',], 
    'contrast': ['ESR_iGuide_contrast_flags','clinician_contrast_flags','chatGPT_contrast_flags','claude_contrast_flags']
}

# Metric functions you care about
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
METRICS = {
    'accuracy':  accuracy_score,
    'precision': lambda y, p: precision_score(y, p, average='weighted', zero_division=0),
    'recall':    lambda y, p: recall_score(y, p, average='weighted', zero_division=0),
    'f1':        lambda y, p: f1_score(y, p, average='weighted', zero_division=0),
}

# --------------------------------------------------------------
# 1 .  Slice the two modality subsets *once*
# --------------------------------------------------------------
ct_df  = data[data['sex'] == 'm']
ang_df = data[data['sex'] == 'f']

if ct_df.empty or ang_df.empty:
    raise ValueError("Need at least one CT and one angiography row.")

# --------------------------------------------------------------
# 2 .  Permutation helper (unchanged)
# --------------------------------------------------------------
def permutation_gap_pvalue(y_true_a, y_pred_a,
                           y_true_b, y_pred_b,
                           metric_func, n_permutations=1000):
    m_a = (y_true_a != -1) & (y_pred_a != -1)
    m_b = (y_true_b != -1) & (y_pred_b != -1)
    y_true_a, y_pred_a = y_true_a[m_a], y_pred_a[m_a]
    y_true_b, y_pred_b = y_true_b[m_b], y_pred_b[m_b]

    if len(y_true_a) == 0 or len(y_true_b) == 0:
        return np.nan

    obs_gap = metric_func(y_true_b, y_pred_b) - metric_func(y_true_a, y_pred_a)

    y_true_pool = np.concatenate([y_true_a, y_true_b])
    y_pred_pool = np.concatenate([y_pred_a, y_pred_b])
    n_a = len(y_true_a)

    hits = 0
    for _ in range(n_permutations):
        perm_idx = np.random.permutation(len(y_true_pool))
        a_idx, b_idx = perm_idx[:n_a], perm_idx[n_a:]
        perm_gap = (metric_func(y_true_pool[b_idx], y_pred_pool[b_idx]) -
                    metric_func(y_true_pool[a_idx], y_pred_pool[a_idx]))
        if abs(perm_gap) >= abs(obs_gap):
            hits += 1
    return hits / n_permutations

# --------------------------------------------------------------
# 3 .  Run for every model × metric
# --------------------------------------------------------------
# --------------------------------------------------------------
# 3 .  Run for every entry (exam / organ / contrast) × model × metric
# --------------------------------------------------------------
rows = []
N_PERM = 1_000           # permutations; raise if you need more precision

for entry_name, cols in entries.items():
    y_true_col  = cols[0]     # ESR_iGuide_* column
    model_cols  = cols[1:]    # clinician / chatGPT / claude

    for pred_col in model_cols:
        # Make a short model label: clinician_exam_flags ➜ clinician, etc.
        model_name = pred_col.split('_')[0]

        for metric_name, metric_func in METRICS.items():   # <-- .items() !!
            p_val = permutation_gap_pvalue(
                ct_df[y_true_col],  ct_df[pred_col],
                ang_df[y_true_col], ang_df[pred_col],
                metric_func=metric_func,
                n_permutations=N_PERM
            )
            rows.append({
                'entry':   entry_name,                 # exam / organ / contrast
                'llm':     model_name,                 # clinician / chatgpt / claude
                'metric':  metric_name,                # accuracy / precision / …
                'p_value_sex': p_val,
                'n_CT':    len(ct_df),
                'n_angio': len(ang_df),
            })

overall_gap_df = pd.DataFrame(rows)
print(overall_gap_df.head())

# optional: save
# overall_gap_df.to_excel(r"C:\work\...\CT_vs_angio_all_models.xlsx", index=False)

# optional: save
# overall_gap_df.to_excel(r"C:\work\...\CT_vs_angio_all_models.xlsx", index=False)


In [ ]:
#### by age

In [ ]:

# --- Two simple age bands ----------------------------------------------------
cuts   = [0, 65, np.inf]        # 0 ≤ age < 65  |  65 ≤ age
labels = ['<65','65<']         # or use '≤64' / '65+' if you prefer

data['age_group'] = pd.cut(
    data['Patient Age'].astype(float),  # make sure it’s numeric
    bins=cuts,
    labels=labels,
    right=False,        # left-closed, right-open intervals
    include_lowest=True # include age = 0 if you ever have it
)

# Quick sanity check
print(data['age_group'].value_counts(dropna=False))

In [ ]:
# --------------------------------------------------------------
# 0 .  Setup  (make sure these columns exist in exam_data)
# --------------------------------------------------------------
entries = {
    'exam': ['ESR_iGuide_exam_flags','clinician_exam_flags','chatGPT_exam_flags','claude_exam_flags'], 
    'organ':['ESR_iGuide_organ_flags','clinician_organ_flags','chatGPT_organ_flags','claude_organ_flags',], 
    'contrast': ['ESR_iGuide_contrast_flags','clinician_contrast_flags','chatGPT_contrast_flags','claude_contrast_flags']
}

# Metric functions you care about
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
METRICS = {
    'accuracy':  accuracy_score,
    'precision': lambda y, p: precision_score(y, p, average='weighted', zero_division=0),
    'recall':    lambda y, p: recall_score(y, p, average='weighted', zero_division=0),
    'f1':        lambda y, p: f1_score(y, p, average='weighted', zero_division=0),
}

# --------------------------------------------------------------
# 1 .  Slice the two modality subsets *once*
# --------------------------------------------------------------
ct_df  = data[data['age_group'] == '65<']
ang_df = data[data['age_group'] == '<65']

if ct_df.empty or ang_df.empty:
    raise ValueError("Need at least one CT and one angiography row.")

# --------------------------------------------------------------
# 2 .  Permutation helper (unchanged)
# --------------------------------------------------------------
def permutation_gap_pvalue(y_true_a, y_pred_a,
                           y_true_b, y_pred_b,
                           metric_func, n_permutations=1000):
    m_a = (y_true_a != -1) & (y_pred_a != -1)
    m_b = (y_true_b != -1) & (y_pred_b != -1)
    y_true_a, y_pred_a = y_true_a[m_a], y_pred_a[m_a]
    y_true_b, y_pred_b = y_true_b[m_b], y_pred_b[m_b]

    if len(y_true_a) == 0 or len(y_true_b) == 0:
        return np.nan

    obs_gap = metric_func(y_true_b, y_pred_b) - metric_func(y_true_a, y_pred_a)

    y_true_pool = np.concatenate([y_true_a, y_true_b])
    y_pred_pool = np.concatenate([y_pred_a, y_pred_b])
    n_a = len(y_true_a)

    hits = 0
    for _ in range(n_permutations):
        perm_idx = np.random.permutation(len(y_true_pool))
        a_idx, b_idx = perm_idx[:n_a], perm_idx[n_a:]
        perm_gap = (metric_func(y_true_pool[b_idx], y_pred_pool[b_idx]) -
                    metric_func(y_true_pool[a_idx], y_pred_pool[a_idx]))
        if abs(perm_gap) >= abs(obs_gap):
            hits += 1
    return hits / n_permutations

# --------------------------------------------------------------
# 3 .  Run for every model × metric
# --------------------------------------------------------------
# --------------------------------------------------------------
# 3 .  Run for every entry (exam / organ / contrast) × model × metric
# --------------------------------------------------------------
rows = []
N_PERM = 1_000           # permutations; raise if you need more precision

for entry_name, cols in entries.items():
    y_true_col  = cols[0]     # ESR_iGuide_* column
    model_cols  = cols[1:]    # clinician / chatGPT / claude

    for pred_col in model_cols:
        # Make a short model label: clinician_exam_flags ➜ clinician, etc.
        model_name = pred_col.split('_')[0]

        for metric_name, metric_func in METRICS.items():   # <-- .items() !!
            p_val = permutation_gap_pvalue(
                ct_df[y_true_col],  ct_df[pred_col],
                ang_df[y_true_col], ang_df[pred_col],
                metric_func=metric_func,
                n_permutations=N_PERM
            )
            rows.append({
                'entry':   entry_name,                 # exam / organ / contrast
                'llm':     model_name,                 # clinician / chatgpt / claude
                'metric':  metric_name,                # accuracy / precision / …
                'p_value_age_cut_65': p_val,
                'n_CT':    len(ct_df),
                'n_angio': len(ang_df),
            })

overall_gap_df = pd.DataFrame(rows)
print(overall_gap_df.head())

# optional: save
# overall_gap_df.to_excel(r"C:\work\...\CT_vs_angio_all_models.xlsx", index=False)

# optional: save
# overall_gap_df.to_excel(r"C:\work\...\CT_vs_angio_all_models.xlsx", index=False)


In [ ]:
# ------------------------------------------------------------------
# 0.  Setup (unchanged)
# ------------------------------------------------------------------
entries = {
    'exam':     ['ESR_iGuide_exam_flags',
                 'clinician_exam_flags', 'chatGPT_exam_flags', 'claude_exam_flags'],
    'organ':    ['ESR_iGuide_organ_flags',
                 'clinician_organ_flags', 'chatGPT_organ_flags', 'claude_organ_flags'],
    'contrast': ['ESR_iGuide_contrast_flags',
                 'clinician_contrast_flags', 'chatGPT_contrast_flags', 'claude_contrast_flags']
}

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
METRICS = {
    'accuracy':  accuracy_score,
    'precision': lambda y, p: precision_score(y, p, average='weighted', zero_division=0),
    'recall':    lambda y, p: recall_score(y, p, average='weighted', zero_division=0),
    'f1':        lambda y, p: f1_score(y, p, average='weighted', zero_division=0),
}

SEX_COL   = "sex"          # adjust if your column is named differently
N_PERM    = 1_000          # raise for finer p-value resolution

# ------------------------------------------------------------------
# 1.  Permutation helper (unchanged)
# ------------------------------------------------------------------
def permutation_gap_pvalue(y_true_a, y_pred_a,
                           y_true_b, y_pred_b,
                           metric_func, n_permutations=N_PERM):
    m_a = (y_true_a != -1) & (y_pred_a != -1)
    m_b = (y_true_b != -1) & (y_pred_b != -1)
    y_true_a, y_pred_a = y_true_a[m_a], y_pred_a[m_a]
    y_true_b, y_pred_b = y_true_b[m_b], y_pred_b[m_b]

    if len(y_true_a) == 0 or len(y_true_b) == 0:
        return np.nan

    obs_gap = metric_func(y_true_b, y_pred_b) - metric_func(y_true_a, y_pred_a)

    y_true_pool = np.concatenate([y_true_a, y_true_b])
    y_pred_pool = np.concatenate([y_pred_a, y_pred_b])
    n_a = len(y_true_a)

    hits = 0
    for _ in range(n_permutations):
        perm_idx = np.random.permutation(len(y_true_pool))
        a_idx, b_idx = perm_idx[:n_a], perm_idx[n_a:]
        perm_gap = (metric_func(y_true_pool[b_idx], y_pred_pool[b_idx]) -
                    metric_func(y_true_pool[a_idx], y_pred_pool[a_idx]))
        if abs(perm_gap) >= abs(obs_gap):
            hits += 1
    return hits / n_permutations

# ------------------------------------------------------------------
# 2.  Female vs Male  inside each test-type
# ------------------------------------------------------------------
rows = []

for test_name, test_df in data.groupby('test_type', observed=True):
    fem_df  = test_df[test_df[SEX_COL] == 'f']
    male_df = test_df[test_df[SEX_COL] == 'm']
    if fem_df.empty or male_df.empty:        # need both sexes present
        continue

    for entry_name, cols in entries.items():
        y_true_col = cols[0]
        model_cols = cols[1:]

        for pred_col in model_cols:
            model_name = pred_col.split('_')[0]   # clinician / chatGPT / claude

            for metric_name, metric_func in METRICS.items():
                p_val = permutation_gap_pvalue(
                    fem_df[y_true_col],  fem_df[pred_col],
                    male_df[y_true_col], male_df[pred_col],
                    metric_func=metric_func
                )
                rows.append({
                    'test_type': test_name,            # ct / angiography / …
                    'entry':     entry_name,           # exam / organ / contrast
                    'llm':       model_name,           # clinician / chatgpt / claude
                    'metric':    metric_name,          # accuracy / precision / …
                    'p_value_female_vs_male': p_val,
                    'n_female':  len(fem_df),
                    'n_male':    len(male_df),
                })

overall_gap_df = pd.DataFrame(rows)
print(overall_gap_df.head())

# optional:
# overall_gap_df.to_excel(r"C:\work\...\female_vs_male_all_models.xlsx", index=False)


In [ ]:
# ------------------------------------------------------------------
# 0.  Setup (unchanged)
# ------------------------------------------------------------------
entries = {
    'exam':     ['ESR_iGuide_exam_flags',
                 'clinician_exam_flags', 'chatGPT_exam_flags', 'claude_exam_flags'],
    'organ':    ['ESR_iGuide_organ_flags',
                 'clinician_organ_flags', 'chatGPT_organ_flags', 'claude_organ_flags'],
    'contrast': ['ESR_iGuide_contrast_flags',
                 'clinician_contrast_flags', 'chatGPT_contrast_flags', 'claude_contrast_flags']
}

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
METRICS = {
    'accuracy':  accuracy_score,
    'precision': lambda y, p: precision_score(y, p, average='weighted', zero_division=0),
    'recall':    lambda y, p: recall_score(y, p, average='weighted', zero_division=0),
    'f1':        lambda y, p: f1_score(y, p, average='weighted', zero_division=0),
}

SEX_COL   = "age_group"          # adjust if your column is named differently
N_PERM    = 1_000          # raise for finer p-value resolution

# ------------------------------------------------------------------
# 1.  Permutation helper (unchanged)
# ------------------------------------------------------------------
def permutation_gap_pvalue(y_true_a, y_pred_a,
                           y_true_b, y_pred_b,
                           metric_func, n_permutations=N_PERM):
    m_a = (y_true_a != -1) & (y_pred_a != -1)
    m_b = (y_true_b != -1) & (y_pred_b != -1)
    y_true_a, y_pred_a = y_true_a[m_a], y_pred_a[m_a]
    y_true_b, y_pred_b = y_true_b[m_b], y_pred_b[m_b]

    if len(y_true_a) == 0 or len(y_true_b) == 0:
        return np.nan

    obs_gap = metric_func(y_true_b, y_pred_b) - metric_func(y_true_a, y_pred_a)

    y_true_pool = np.concatenate([y_true_a, y_true_b])
    y_pred_pool = np.concatenate([y_pred_a, y_pred_b])
    n_a = len(y_true_a)

    hits = 0
    for _ in range(n_permutations):
        perm_idx = np.random.permutation(len(y_true_pool))
        a_idx, b_idx = perm_idx[:n_a], perm_idx[n_a:]
        perm_gap = (metric_func(y_true_pool[b_idx], y_pred_pool[b_idx]) -
                    metric_func(y_true_pool[a_idx], y_pred_pool[a_idx]))
        if abs(perm_gap) >= abs(obs_gap):
            hits += 1
    return hits / n_permutations

# ------------------------------------------------------------------
# 2.  Female vs Male  inside each test-type
# ------------------------------------------------------------------
rows = []

for test_name, test_df in data.groupby('test_type', observed=True):
    fem_df  = test_df[test_df[SEX_COL] == '<65']
    male_df = test_df[test_df[SEX_COL] == '65<']
    if fem_df.empty or male_df.empty:        # need both sexes present
        continue

    for entry_name, cols in entries.items():
        y_true_col = cols[0]
        model_cols = cols[1:]

        for pred_col in model_cols:
            model_name = pred_col.split('_')[0]   # clinician / chatGPT / claude

            for metric_name, metric_func in METRICS.items():
                p_val = permutation_gap_pvalue(
                    fem_df[y_true_col],  fem_df[pred_col],
                    male_df[y_true_col], male_df[pred_col],
                    metric_func=metric_func
                )
                rows.append({
                    'test_type': test_name,            # ct / angiography / …
                    'entry':     entry_name,           # exam / organ / contrast
                    'llm':       model_name,           # clinician / chatgpt / claude
                    'metric':    metric_name,          # accuracy / precision / …
                    'p_value_age_group': p_val,
                    'n_female':  len(fem_df),
                    'n_male':    len(male_df),
                })

overall_gap_df = pd.DataFrame(rows)
print(overall_gap_df.head())

# optional:
# overall_gap_df.to_excel(r"C:\work\...\female_vs_male_all_models.xlsx", index=False)


In [ ]:



sns.set_style("whitegrid")
RNG      = np.random.default_rng(seed=42)   # reproducible bootstrap
N_BOOT   = 1_000                            # raise for tighter CIs
ALPHA    = 0.95
OUT_DIR  = Path
OUT_DIR.mkdir(parents=True, exist_ok=True)

# --------------------------------------------------------------
# 1.  Column groups and metric dict
# --------------------------------------------------------------
entries = {
    'exam'    : ['ESR_iGuide_exam_flags','clinician_exam_flags','chatGPT_exam_flags','claude_exam_flags'],
    'organ'   : ['ESR_iGuide_organ_flags','clinician_organ_flags','chatGPT_organ_flags','claude_organ_flags'],
    'contrast': ['ESR_iGuide_contrast_flags','clinician_contrast_flags','chatGPT_contrast_flags','claude_contrast_flags'],
}

METRICS = {
    'accuracy' : accuracy_score,
    'precision': lambda y, p: precision_score(y, p, average='weighted', zero_division=0),
    'recall'   : lambda y, p: recall_score(y, p, average='weighted', zero_division=0),
    'f1'       : lambda y, p: f1_score(y, p, average='weighted', zero_division=0),
}

# --------------------------------------------------------------
# 2.  Helper to get point estimate + bootstrap CI
# --------------------------------------------------------------
def metric_with_ci(y_true, y_pred, metric_func, n_boot=N_BOOT, alpha=ALPHA):
    point = metric_func(y_true, y_pred)
    if n_boot == 0:
        return point, np.nan, np.nan

    boot_stats = []
    n = len(y_true)
    for _ in range(n_boot):
        idx = RNG.integers(0, n, n)        # sample w. replacement
        boot_stats.append(metric_func(y_true[idx], y_pred[idx]))
    low, high = np.percentile(boot_stats, [(1-alpha)/2*100, (1+(alpha))/2*100])
    return point, low, high

# --------------------------------------------------------------
# 3-A.  CT  vs  CT-angiography ─ metrics & CIs
# --------------------------------------------------------------
ct_df  = data.loc[data['test_type'] == 'ct']
cta_df = data.loc[data['test_type'] == 'angiography']
if ct_df.empty or cta_df.empty:
    raise ValueError("Need at least one CT and one angiography row.")

rows_test = []
for entry_name, cols in entries.items():
    y_true_col  = cols[0]        # ESR ground-truth flags
    for pred_col in cols[1:]:
        model = pred_col.split('_')[0]
        for m_name, m_func in METRICS.items():
            # CT
            est_ct, lo_ct, hi_ct = metric_with_ci(
                ct_df[y_true_col].values,  ct_df[pred_col].values,  m_func
            )
            # CTA
            est_cta, lo_cta, hi_cta = metric_with_ci(
                cta_df[y_true_col].values, cta_df[pred_col].values, m_func
            )

            rows_test.append({
                'entry'   : entry_name,
                'model'   : model,
                'metric'  : m_name,
                'group'   : 'CT',           'est' : est_ct,  'ci_low': lo_ct,  'ci_high': hi_ct,
            })
            rows_test.append({
                'entry'   : entry_name,
                'model'   : model,
                'metric'  : m_name,
                'group'   : 'CTA',          'est' : est_cta, 'ci_low': lo_cta, 'ci_high': hi_cta,
            })

df_test = pd.DataFrame(rows_test)
df_test.to_excel(OUT_DIR / "metrics_CT_vs_CTA.xlsx", index=False)

# --------------------------------------------------------------
# 3-B.  Age (<65  vs  ≥65) ─ metrics & CIs
# --------------------------------------------------------------
age_df = data.copy()
age_df['age_bin'] = np.where(age_df['Patient Age'] < 65, '<65', '≥65')

rows_age = []
for entry_name, cols in entries.items():
    y_true_col  = cols[0]
    for pred_col in cols[1:]:
        model = pred_col.split('_')[0]
        for m_name, m_func in METRICS.items():
            for grp, g in age_df.groupby('age_bin'):
                est, lo, hi = metric_with_ci(g[y_true_col].values, g[pred_col].values, m_func)
                rows_age.append({
                    'entry': entry_name, 'model': model, 'metric': m_name,
                    'group': grp, 'est': est, 'ci_low': lo, 'ci_high': hi
                })

df_age = pd.DataFrame(rows_age)
df_age.to_excel(OUT_DIR / "metrics_under65_vs_over65.xlsx", index=False)

# --------------------------------------------------------------
# 4.  FIGURE 1  –  Forest / dot-whisker  (accuracy only, exam-level)
# --------------------------------------------------------------
plot_df = (
    df_test
      .query("entry == 'exam' and metric == 'accuracy'")
      .sort_values(['model', 'group'])
      .reset_index(drop=True)          #  ← this line
)

fig, ax = plt.subplots(figsize=(6, 3.5))
y_pos = np.arange(len(plot_df))
colors = {'CT':'black', 'CTA':'slategray'}

for i, row in plot_df.iterrows():
    ax.errorbar(row['est']*100,               # convert to %
                y_pos[i],
                xerr=[[ (row['est']-row['ci_low'])*100 ],
                      [ (row['ci_high']-row['est'])*100 ]],
                fmt='o', color=colors[row['group']],
                capsize=3, markersize=5, label=row['group'] if i < 2 else "")

ax.set_yticks(y_pos)
ax.set_yticklabels([f"{r['model']}" for _, r in plot_df.iterrows()])
ax.set_xlabel("Accuracy (%)")
ax.set_title("Figure 1. CT vs CT-angiography — exam-level accuracy")
ax.invert_yaxis()
ax.legend(title="Modality", frameon=False, loc='lower right')
plt.tight_layout()
plt.savefig(OUT_DIR / "Fig1_forest_CT_vs_CTA_accuracy.png", dpi=600)
plt.close()

# --------------------------------------------------------------
# 5.  FIGURE 2  –  Grouped bar + CIs (accuracy, exam-level, <65 vs ≥65)
# --------------------------------------------------------------
plot_df2 = (
    df_age
    .query("entry == 'exam' and metric == 'accuracy'")
    .pivot_table(index=['model','group'], values=['est','ci_low','ci_high'])
    .reset_index()
)

fig, ax = plt.subplots(figsize=(6, 3.5))
x = np.arange(len(plot_df2['model'].unique()))
bar_w = 0.35

for j, grp in enumerate(['<65','≥65']):
    grp_df = plot_df2[plot_df2['group']==grp]
    est_pct = grp_df['est'].values*100
    err_low = (grp_df['est'] - grp_df['ci_low']).values*100
    err_high = (grp_df['ci_high'] - grp_df['est']).values*100
    ax.bar(x + (j-0.5)*bar_w, est_pct, bar_w,
           yerr=[err_low, err_high], capsize=4,
           label=grp, color=('white' if grp=='<65' else 'lightgray'),
           edgecolor='black')

ax.set_xticks(x)
ax.set_xticklabels(grp_df['model'])
ax.set_ylabel("Accuracy (%)")
ax.set_title("Figure 2. Under-65 vs ≥65 — exam-level accuracy")
ax.legend(title="Age group", frameon=False)
plt.tight_layout()
plt.savefig(OUT_DIR / "Fig2_groupedBar_age_accuracy.png", dpi=600)
plt.close()

print("✓  Metrics tables and both figures were saved in", OUT_DIR)


In [ ]:
# ------------------------------------------------------------
#  FIGURE 1  –  one PNG per metric (organ task, CT vs CTA)
# ------------------------------------------------------------
METRICS_TO_PLOT = ["accuracy", "precision", "recall", "f1"]
pretty = {"clinician": "Clinicians", "chatGPT": "GPT-4", "claude": "Claude"}

# tidy once
plot_df_all = (
    df_test
        .query("entry == 'organ' and metric in @METRICS_TO_PLOT")
        .reset_index(drop=True)
)
plot_df_all["pretty_model"] = plot_df_all["model"].map(pretty)

bar_w   = 0.35
ct_col  = "#181C14"   # dark
cta_col = "#7D7C7C"   # light grey

for metric in METRICS_TO_PLOT:
    fig, ax = plt.subplots(figsize=(4, 4))

    sub = plot_df_all.query("metric == @metric")

    for i, mdl in enumerate(["Clinicians", "GPT-4", "Claude"]):
        # ---- CT ----
        ct_row  = sub[(sub.pretty_model == mdl) & (sub.group == "CT")].iloc[0]
        est_ct  = ct_row.est  * 100
        err_ct  = (ct_row.ci_high - ct_row.ci_low) / 2 * 100
        ax.bar(i - bar_w/2, est_ct, bar_w,
               yerr=err_ct, capsize=4, color=ct_col, label="CT" if i==0 else "")

        # ---- CTA ----
        cta_row = sub[(sub.pretty_model == mdl) & (sub.group == "CTA")].iloc[0]
        est_cta = cta_row.est * 100
        err_cta = (cta_row.ci_high - cta_row.ci_low) / 2 * 100
        ax.bar(i + bar_w/2, est_cta, bar_w,
               yerr=err_cta, capsize=4, color=cta_col, label="CTA" if i==0 else "")

    # cosmetics
    ax.set_xticks(range(3))
    ax.set_xticklabels(["Clinicians", "GPT-4", "Claude"], rotation=15)
    ax.set_ylabel("Rate (%)")
    ax.set_ylim(0, 102)
    ax.set_title(f"{metric.capitalize()}")
    if metric == "accuracy":        # legend once (first loop iteration)
        ax.legend(frameon=False, loc="upper left")

    fig.tight_layout()
    plt.grid(False)
    fname = f"Fig1_{metric}_CT_vs_CTA_organ.png"
    fig.savefig(OUT_DIR / fname, dpi=600)
    plt.show(fig)
    print("✓ saved", fname)


In [ ]:


RNG         = np.random.default_rng(42)
N_BOOT      = 1_000
N_PERM      = 1_000
ALPHA       = 0.95

MODELS = {"clinician": "Clinicians",
          "chatGPT"  : "GPT-4",
          "claude"   : "Claude"}

FILE    = Path 
OUT_DIR = Path
OUT_DIR.mkdir(exist_ok=True)

# 2 ── helpers ─────────────────────────────────────────────────
def bootstrap_ci(y, p):
    """return point, ci_lo, ci_hi (as %)"""
    y, p = np.asarray(y), np.asarray(p)
    point = cohen_kappa_score(y, p)
    boot  = [cohen_kappa_score(y[RNG.integers(len(y), size=len(y))],
                               p[RNG.integers(len(p), size=len(p))])
             for _ in range(N_BOOT)]
    lo, hi = np.percentile(boot, [(1-ALPHA)/2*100, (1+ALPHA)/2*100])
    return point*100, lo*100, hi*100

def perm_pvalue(y_ct, p_ct, y_cta, p_cta):
    """two-sided permutation p for κ gap"""
    obs = cohen_kappa_score(y_cta, p_cta) - cohen_kappa_score(y_ct, p_ct)
    pool_y = np.concatenate([y_ct, y_cta])
    pool_p = np.concatenate([p_ct, p_cta])
    n_ct   = len(y_ct)
    hits = 0
    for _ in range(N_PERM):
        idx = RNG.permutation(len(pool_y))
        gap = cohen_kappa_score(pool_y[idx[n_ct:]], pool_p[idx[n_ct:]]) - \
              cohen_kappa_score(pool_y[idx[:n_ct]], pool_p[idx[:n_ct]])
        hits += abs(gap) >= abs(obs)
    return (hits + 1) / (N_PERM + 1)

# 3 ── load data ───────────────────────────────────────────────
df = pd.read_excel(FILE)
mask_ok = lambda y, p: (y != -1) & (p != -1)

# 4 ── compute κ & p-values ────────────────────────────────────
rows = []
for raw, nice in MODELS.items():
    pred_col = f"{raw}_organ_flags"

    # split once
    ct   = df[df["test_type"] == "ct"]
    cta  = df[df["test_type"] == "angiography"]

    ok_ct  = mask_ok(ct["ESR_iGuide_organ_flags"],  ct[pred_col])
    ok_cta = mask_ok(cta["ESR_iGuide_organ_flags"], cta[pred_col])

    # κ + CI for each modality
    for label, data, ok in [("CT", ct, ok_ct), ("CTA", cta, ok_cta)]:
        kappa, lo, hi = bootstrap_ci(data["ESR_iGuide_organ_flags"][ok],
                                     data[pred_col][ok])
        rows.append(dict(model=nice, modality=label,
                         kappa=kappa, ci_lo=lo, ci_hi=hi))

    # permutation p-value for the gap (CTA − CT)
    p_val = perm_pvalue(ct["ESR_iGuide_organ_flags"][ok_ct].values,
                        ct[pred_col][ok_ct].values,
                        cta["ESR_iGuide_organ_flags"][ok_cta].values,
                        cta[pred_col][ok_cta].values)
    rows.append(dict(model=nice, modality="p-value",
                     kappa=p_val))

table = pd.DataFrame(rows)

# 5 ── save ────────────────────────────────────────────────────
table.to_csv(OUT_DIR / "kappa_CT_vs_CTA.csv", index=False)
print("✓  Cohen κ table written to", OUT_DIR / "kappa_CT_vs_CTA.csv")




In [ ]:

RNG, N_BOOT, N_PERM, ALPHA = np.random.default_rng(42), 1_000, 1_000, .95

# --- config ---
METRICS = {"accuracy": accuracy_score,
           "precision": lambda y,p: precision_score(y,p,average="weighted",zero_division=0),
           "recall":    lambda y,p: recall_score(y,p,average="weighted",zero_division=0),
           "f1":        lambda y,p: f1_score(y,p,average="weighted",zero_division=0)}
MODELS  = {"clinician":"Clinicians", "chatGPT":"GPT-4", "claude":"Claude"}
FILE    = Path
OUT_DIR = Path; OUT_DIR.mkdir(exist_ok=True)

# --- helpers ---
def bootstrap_ci(y, p, func, n_boot=N_BOOT, alpha=ALPHA):
    """Return point estimate, low-CI, high-CI (percent scale)."""
    # --- NEW: guarantee positional indexing ---
    y = np.asarray(y)
    p = np.asarray(p)
    # ------------------------------------------
    point = func(y, p)
    boot_stats = []
    n = len(y)
    for _ in range(n_boot):
        idx = RNG.integers(0, n, n)          # sample positions
        boot_stats.append(func(y[idx], p[idx]))
    lo, hi = np.percentile(
        boot_stats, [(1-alpha)/2*100, (1+alpha)/2*100]
    )
    return point*100, lo*100, hi*100


def stars(p): return "★★★" if p<.001 else ("★★" if p<.01 else ("★" if p<.05 else ""))

# --- load & basic wrangling ---
df = pd.read_excel(FILE)
df["age_bin"] = np.where(df["Patient Age"]<65, "<65", "≥65")
mask = lambda y,p: (y != -1) & (p != -1)
row_colors = sns.color_palette('Greys', len(METRICS)) 
# -------------------------------------------------------------
#  Build tidy table for one grouping variable (age OR sex)
# -------------------------------------------------------------
def build_table(group_var, groups):
    recs=[]
    for raw, nice in MODELS.items():
        pred = f"{raw}_organ_flags"
        for mname, mfunc in METRICS.items():
            for grp in groups:
                for sex in ["Female","Male"] if group_var=="Patient Sex" else [grp]:
                    # CT & CTA separately
                    for mod in ["ct","angiography"]:
                        g = df[(df[group_var]==grp) & (df["test_type"]==mod)]
                        good = mask(g["ESR_iGuide_organ_flags"], g[pred])
                        est, lo, hi = bootstrap_ci(g["ESR_iGuide_organ_flags"][good],
                                                   g[pred][good], mfunc)
                        recs.append(dict(group=grp, model=nice, metric=mname,
                                         modality=mod.upper(), est=est, lo=lo, hi=hi))
            # permutation p-value (CT vs CTA) once per group
            for grp in groups:
                g1=df[(df[group_var]==grp)&(df["test_type"]=="ct")]
                g2=df[(df[group_var]==grp)&(df["test_type"]=="angiography")]
                good1, good2 = mask(g1["ESR_iGuide_organ_flags"],g1[pred]), mask(g2["ESR_iGuide_organ_flags"],g2[pred])
                gap=METRICS[mname](g2["ESR_iGuide_organ_flags"][good2],g2[pred][good2]) - \
                    METRICS[mname](g1["ESR_iGuide_organ_flags"][good1],g1[pred][good1])
                pool_y=np.concatenate([g1["ESR_iGuide_organ_flags"][good1], g2["ESR_iGuide_organ_flags"][good2]])
                pool_p=np.concatenate([g1[pred][good1], g2[pred][good2]])
                hits=0
                for _ in range(N_PERM):
                    idx=RNG.permutation(len(pool_y))
                    perm_gap=METRICS[mname](pool_y[idx[len(g1[good1]):]],pool_p[idx[len(g1[good1]):]]) - \
                             METRICS[mname](pool_y[idx[:len(g1[good1])]],pool_p[idx[:len(g1[good1])]])
                    hits += abs(perm_gap)>=abs(gap)
                p=(hits+1)/(N_PERM+1)
                recs.append(dict(group=grp, model=nice, metric=mname,
                                 modality="p", est=p))
    tidy=pd.DataFrame(recs)
    return tidy

age_tbl = build_table("age_bin", ["<65","≥65"])
sex_tbl = build_table("Patient Sex", ["Female","Male"])

# -------------------------------------------------------------
#  Cleveland-dot plot function
# -------------------------------------------------------------


# -------------------------------------------------------------
#  6-BIS. Cleveland-dot plot with custom colours
# -------------------------------------------------------------
MODEL_COLORS = {           # <-- customise here if you like
    "Clinicians": "#404040",   # dark grey
    "GPT-4"     : "#1b7837",   # green
    "Claude"    : "#762a83"    # purple
}
BASE_COLOR = "#d7191c"     # red   (baseline subgroup)
COMP_COLOR = "#2c7bb6"     # blue  (comparison subgroup)

def cleveland(tidy, groups, fname, title,
              base_label=None, comp_label=None,
              use_mod="CT"):
    """
    y-axis = metrics; x-axis = rate (%).
    For each metric we draw three model clusters, each with two dots
    (baseline → red, comparison → blue) connected by a line whose
    colour identifies the model.
    """
    fig, ax = plt.subplots(figsize=(8, 4))
    row_spacing = len(MODELS) + 1
    y_base = np.arange(len(METRICS)) * row_spacing

    for col_idx, (raw, mdl) in enumerate(MODELS.items()):
        y_offsets = y_base + col_idx
        for k, metric in enumerate(METRICS):
            y_start = y_base[k] - 0.5             # y_base: first row index for this metric
            y_end   = y_start + row_spacing       # row_spacing = len(MODELS) + 1
            ax.axhspan(y_start, y_end,
               color=row_colors[k],
               alpha=0.15,                # subtle; bump to 0.25 if needed
               zorder=0)
            # ----- baseline dot (groups[0]) -----
            base_val = tidy.query(
                "metric==@metric & model==@mdl & group==@groups[0] & modality==@use_mod"
            ).est.values[0]
            ax.scatter(base_val, y_offsets[k],
                       s=50, color=BASE_COLOR, zorder=3)

            # ----- comparison dot (groups[1]) -----
            comp_val = tidy.query(
                "metric==@metric & model==@mdl & group==@groups[1] & modality==@use_mod"
            ).est.values[0]
            ax.scatter(comp_val, y_offsets[k],
                       s=50, color=COMP_COLOR, zorder=3)

            # ----- connector line coloured by model -----
            ax.hlines(y_offsets[k],
                      xmin=min(base_val, comp_val),
                      xmax=max(base_val, comp_val),
                      color=MODEL_COLORS[mdl],
                      lw=2, zorder=1)
            #ax.grid(axis="y", color="#eeeeee", linewidth=1, zorder=0)   # light grey
            #ax.set_axisbelow(True)
            # ----- significance star next to comparison dot -----
            p_val = tidy.query(
                "metric==@metric & model==@mdl & group==@groups[1] & modality=='p'"
            ).est.values[0]
            if p_val < 0.05:
                star = "***" if p_val < .001 else ("**" if p_val < .01 else "*")
                mid_x = (base_val + comp_val) / 2
                ax.text(mid_x, y_offsets[k], star,
                        va="center", fontsize=11, color="black")

    # aesthetics -------------------------------------------------
    ax.set_yticks(y_base + (len(MODELS)-1)/2)
    ax.set_yticklabels([m.capitalize() for m in METRICS])
    ax.set_xlim(60, 100)
    ax.set_xlabel("Rate (%)")
    ax.set_title(title, loc="left")
    ax.invert_yaxis()
    ax.grid(axis='x', alpha=.2)

    # legends
    from matplotlib.lines import Line2D
    model_legend = [Line2D([0], [0], color=col, lw=3, label=mdl)
                    for mdl, col in MODEL_COLORS.items()]
    dot_legend   = [Line2D([0], [0], marker='o', color='w',
                           markerfacecolor=BASE_COLOR, markersize=8,
                           label=base_label or groups[0]),
                    Line2D([0], [0], marker='o', color='w',
                           markerfacecolor=COMP_COLOR, markersize=8,
                           label=comp_label or groups[1])]
    leg1 = ax.legend(handles=model_legend, title="Model", loc="lower right",
                     frameon=False)
    ax.add_artist(leg1)
    ax.legend(handles=dot_legend, title="Sub-group", loc="upper right",
              frameon=True)

    plt.tight_layout()
    plt.savefig(OUT_DIR / fname, dpi=600)
    plt.show()

# -------------------------------------------------------------
# 7-bis.  Render the Cleveland plots with colours
# -------------------------------------------------------------
cleveland(age_tbl,
          ["<65", "≥65"],
          "Figure3A_CT_age_cleveland.png",
          "A",
          base_label="< 65", comp_label="≥ 65",
          use_mod="CT") 
cleveland(sex_tbl,
          ["Male", "Female"],
          "Figure2A_CT_sex_cleveland.png",
          "A",
          base_label="Male", comp_label="Female",
          use_mod="CT")
cleveland(age_tbl,
          ["<65", "≥65"],
          "Figure3B_CTA_age_cleveland.png",
          "B",
          base_label="< 65", comp_label="≥ 65",
          use_mod="ANGIOGRAPHY")

cleveland(sex_tbl,
          ["Male", "Female"],
          "Figure2B_CTA_sex_cleveland.png",
          "B",
          base_label="Male", comp_label="Female",
          use_mod="ANGIOGRAPHY")


print("✓ colour-coded Cleveland plots saved to", OUT_DIR)

